# Прогнозирование заказов такси

**Описание проекта**

Компания «Чётенькое такси» собрала исторические данные о заказах такси в аэропортах. Чтобы привлекать больше водителей в период пиковой нагрузки, нужно спрогнозировать количество заказов такси на следующий час. Постройте модель для такого предсказания.

Значение метрики RMSE на тестовой выборке должно быть не больше 48.

**Описание данных**

Данные лежат в файле `/datasets/taxi.csv`.

Количество заказов находится в столбце `num_orders`.

In [3]:
import pandas as pd
import optuna
import plotly.express as px

from caseconverter import snakecase
from ydata_profiling import ProfileReport
from IPython.display import display

from fast_ml import eda

from sklearn.model_selection import train_test_split
from sklearn.utils import resample

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

from sklearn.metrics import mean_squared_error

In [4]:
FIG_WIDTH = 10 * 100
FIG_HEIGHT = 5 * 100
RANDOM_SEED = 42

In [5]:
try:
    raw_taxi_orders = pd.read_csv('taxi.csv')
except:
    raw_taxi_orders = pd.read_csv('/datasets/taxi.csv')

## Исследовательский анализ данных

Изучим основные зависимости в данных перед тем, как мы будем использовать их в алгоритмах машинного обучения.

Таблица-резюме:

In [6]:
display(eda.df_info(raw_taxi_orders))

,data_type,data_type_grp,num_unique_values,sample_unique_values,num_missing,perc_missing
datetime,object,Categorical,26496,"[2018-03-01 00:00:00, 2018-03-01 00:10:00, 201...",0,0.0
num_orders,int64,Numerical,81,"[9, 14, 28, 20, 32, 21, 7, 5, 17, 12]",0,0.0


Числовые распределения:

In [7]:
display(round(raw_taxi_orders.describe().T, 2))

,count,mean,std,min,25%,50%,75%,max
num_orders,26496.0,14.07,9.21,0.0,8.0,13.0,19.0,119.0


И детальный отчет:

In [8]:
ProfileReport(raw_taxi_orders).to_widgets()

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]